# Classifying Traffic Signs with OpenCV

For this project we will be trying split the traffic sign dataset into 3 separate classes:
- Stop signs
- Red Circles
- Blue rectangles/squares 

### Imports

In [30]:
import cv2 as cv
import numpy as np
import os, os.path
import shutil
from ImageList import ImageList
from Image import Image
import logging

In [31]:
# Results directories reset
shutil.rmtree('results/blue_squares')
shutil.rmtree('results/red_circles')
shutil.rmtree('results/stop_signs')
shutil.rmtree('results/unknowns')
os.mkdir('results/blue_squares')
os.mkdir('results/red_circles')
os.mkdir('results/stop_signs')
os.mkdir('results/unknowns')

### Useful functions

In [32]:
def get_image_list():
    # Saves images into a list containing objects of class Image which contains traffic sign info 
    image_list = ImageList() 
    for root, _, files in os.walk('./dataset'):
        for file in files:
            split_file_path = os.path.join(root, file).split(os.sep)
            filename = split_file_path[-1]
            _type = split_file_path[-2][:-1]
            # Prevent image duplicates
            idx = image_list.contains_img(filename) 
            if idx >= 0:
                print(123)
                image = image_list.get_image(idx)
                image.add_type(_type)
            else:
                image = Image(filename, _type)
                image_list.add_image(image)
    return image_list

In [33]:
def process_image(segmented_img, original_img, color):
    segmented_img = smooth(segmented_img)
    #_, segmented_img = cv.threshold(segmented_img,1,255,cv.THRESH_BINARY)
    
    if color == "blue":
        kernel = np.ones((5,5), np.uint8)
        segmented_img = cv.dilate(segmented_img, kernel, iterations=3)
        segmented_img = cv.erode(segmented_img, kernel, iterations=3)
        segmented_img = cv.Canny(segmented_img, 150, 200) 
        
    final, prediction = search_contours(segmented_img, original_img.copy(), color)
    
    if type(prediction) != str: # Prediction is returned as a number if found contour has more than 8 edges
        final, prediction = find_circles(segmented_img, original_img.copy(), int(prediction))
        # if prediction == 'unrecognized':
        #     final, prediction = find_circles_possibly(segmented_img, original_img, color)
        
    return final, prediction

In [34]:
# Remove image noise with gaussian blur (better at preserving edges)
def smooth2(img):
    return cv.GaussianBlur(img, (3,3), 0)

def smooth(img):
    return cv.bilateralFilter(img, 5, 75, 75)

In [35]:
# Increases image brightness
def increase_brightness(img, value=30):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv.merge((h, s, v))
    img = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    return img

In [36]:
#  Segments reds and blues of an image
def segment(img, color):
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    if color == 'red':
        # lower mask (0-10)
        lower_red = np.array([0,90,50])
        upper_red = np.array([10,255,255])
        mask0 = cv.inRange(img_hsv, lower_red, upper_red)  

        # upper mask (160-180)
        lower_red = np.array([160,90,50])
        upper_red = np.array([180,255,255])
        mask1 = cv.inRange(img_hsv, lower_red, upper_red)

        mask = mask0 | mask1
    elif color == 'blue':
        lower_blue = np.array([90,140,50])
        upper_blue = np.array([130,255,255])
        mask = cv.inRange(img_hsv, lower_blue, upper_blue) 

    segmented_img = cv.bitwise_and(img_hsv, img_hsv, mask = mask)
    segmented_img = cv.cvtColor(segmented_img, cv.COLOR_BGR2GRAY)
    return segmented_img    


In [37]:
# Find the contour with the largest area in a gray scale image
def find_largest_contour(segmented_img, original_img):
    contours, _ = cv.findContours(segmented_img,2,1)
    cnt = contours
    big_contour = []
    max = 0
    for i in cnt:
        area = cv.contourArea(i)
        if(area > max):
            max = area
            big_contour = i 
    if max > 1000 and len(big_contour) > 10 and len(big_contour) < 1200:
        final = cv.drawContours(original_img, big_contour, -1, (0,255,0), 3)
    else:
        final = original_img

    return final, big_contour

In [38]:
# Find a blue square
def search_contours(segmented_img, original_img, color):
    if cv.countNonZero(segmented_img) == 0:
        return original_img, 'unrecognised'
    final, contour = find_largest_contour(segmented_img, original_img)
    if len(contour) > 100:
        peri = cv.arcLength(contour, True)
        approx = cv.approxPolyDP(contour, 0.01 * peri, True)
        if len(approx) > 8: # circle possibly
            return final, cv.contourArea(contour)
        return final, classify_sign(approx, color)

    return final, 'unrecognised'

In [39]:
def classify_sign(approx_cnt, color):
    if color == 'blue':
        if len(approx_cnt) == 4:
            return 'blue_square'
    elif color == 'red':
        if len(approx_cnt) == 8:
            return 'stop_sign'
    return 'unrecognised'

In [40]:
def find_circles(img, original_img, radius):
    # Reduce white noise and enhance shapes
    kernel = np.ones((3, 3), np.uint8)
    img = cv.dilate(img, kernel, iterations = 3)
    img = cv.erode(img, kernel, iterations = 3)
    #Tolerance
    tol = 5

    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, 1, radius * 2 - tol, param1=100, param2=45, minRadius=5, maxRadius=radius + tol)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0, :]:
            center = (i[0], i[1])
            r = i[2]
            cv.circle(original_img, center, r, (0, 255, 0), 2)
        return original_img, 'red_circle'
    else:
        return original_img, 'unrecognised'


## Red Circles

This is the section to detect red circles

After processing the image to gray values holding as the white value the red features a contour finder is used to detect an ellipse correctly. It can also be used in the case of stop signs to obtain octagons.

#### Paths and image list

In [41]:
dataset_path = 'dataset'
results_path = 'results'

# Image List
img_list = get_image_list()

### More Smoothing approach

In order to reduce the amount of unrecognised traffic signs in the dataset images we tried to increase images smoothing until the results started getting worse. Thats why the "amount" of smoothing done in original images for blue segmented image processing is higher than the "amount" used in original images for red segmented images. 

### TODO METER EXEMPLOS AQUI DO EFEITO DO SMOOTHING EM DIFERENTES IMAGENS

### Different Smoothing Amount Solutions Comparison

Different amount of bilateral (filter) smoothing applied in original images before segmenting 

In [42]:
# Counters
cnt_un1 = 0    # Counter for the amount of images unrecognised prior to using more smoothing if it is unrecognised
cnt_un2 = 0      # Counter for the amount of images unrecognised after using more smoothing
cnt_un3 = 0      # Counter for the amount of images unrecognised after using no smoothing

# Accuracies before using lower amount of smoothing
red_circles_accuracy1 = 0    # Red circles classification accuracy
blue_squares_accuracy1 = 0    # Blue squares classification accuracy
stop_signs_accuracy1 = 0    # Stop signs classification accuracy

# Accuracies after using higher amount of smoothing
red_circles_accuracy2 = 0    # Red circles classification accuracy
blue_squares_accuracy2 = 0    # Blue squares classification accuracy
stop_signs_accuracy2 = 0    # Stop signs classification accuracy

# Accuracies after using no smoothing
red_circles_accuracy3 = 0    # Red circles classification accuracy
blue_squares_accuracy3 = 0    # Blue squares classification accuracy
stop_signs_accuracy3 = 0    # Stop signs classification accuracy

In [48]:
inp = 0
while inp <= 0 or inp > 3:
    inp = int(input('Select solution to be written to the results directory (1- Normal Smoothing; 2- High Smoothing; 3- No Smoothing): '))

#### Normal amount

In [49]:
for i in range(0, img_list.len()):
    # Get img object from img_list
    img_obj = img_list.get_image(i)

    # Get path of that image -> {type(qualquer um serve)}/{filename}
    img_path = img_obj.types[0] + 's' + os.sep + img_obj.filename

    # Image in the dataset path
    img_path_in_dataset = dataset_path + os.sep + img_path

    # Read img
    img = cv.imread(img_path_in_dataset)

    # Keep an original copy
    original = img.copy()

    # Smooth image
    img = smooth(img)

    # Segment it by filtering the blue and red color for blue and red signs
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')

    # Variable to check if any sign is already detected for it not to be classified as unrecognised in that case
    sign_detected = False

    # Process blue segmented img
    final, classification = process_image(img_blue.copy(), img.copy(), 'blue')

    if classification == 'blue_square':
        img_obj.add_classification(classification)
        if inp == 1:
            cv.imwrite(os.path.join(results_path, classification +
                                    's', img_obj.filename), final)
        sign_detected = True

    final, classification = process_image(img_red.copy(), img.copy(), 'red')
    # Process first iteration result
    if classification == 'stop_sign' or classification == 'red_circle':
        img_obj.add_classification(classification)
        if inp == 1:
            cv.imwrite(os.path.join(
                results_path, classification + 's', img_obj.filename), final)
        sign_detected = True

    if sign_detected == False:
        cnt_un1 += 1
        img_obj.add_classification(classification)
        if inp == 1:
            cv.imwrite(os.path.join(
                results_path, 'unknowns', img_obj.filename), final)

#### Higher amount

In [45]:
for i in range(0, img_list.len()):
    # Get img object from img_list
    img_obj = img_list.get_image(i)

    # Get path of that image -> {type(qualquer um serve)}/{filename}
    img_path = img_obj.types[0] + 's' + os.sep + img_obj.filename

    # Image in the dataset path
    img_path_in_dataset = dataset_path + os.sep + img_path

    # Read img
    img = cv.imread(img_path_in_dataset)

    # Keep an original copy
    original = img.copy()

    # Smooth image
    img = smooth(img)

    # Segment it by filtering the blue and red color for blue and red signs
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')

    # Variable to check if any sign is already detected for it not to be classified as unrecognised in that case
    sign_detected = False

    # Process blue segmented img
    final, classification = process_image(img_blue.copy(), img.copy(), 'blue')

    # Process first iteration result
    if classification == 'blue_square':
        img_obj.add_classification(classification)
        if inp == 2:
            cv.imwrite(os.path.join(results_path, classification +
                                    's', img_obj.filename), final)
        sign_detected = True
    elif classification == 'unrecognised':  # if no sign detected do more smoothing
        temp = smooth(smooth(smooth(img_blue)))
        final, classification = process_image(temp, img.copy(), 'blue')
        if classification == 'blue_square':
            img_obj.add_classification(classification)
            if inp == 2:
                cv.imwrite(os.path.join(results_path, classification +
                                        's', img_obj.filename), final)
            sign_detected = True

    # Process red segmented img
    final, classification = process_image(img_red.copy(), img.copy(), 'red')

    # Process first iteration result
    if classification == 'stop_sign' or classification == 'red_circle':
        img_obj.add_classification(classification)
        if inp == 2:
            cv.imwrite(os.path.join(
                results_path, classification + 's', img_obj.filename), final)
        sign_detected = True
    elif classification == 'unrecognised':  # if no sign detected do more smoothing
        temp = smooth(img_red)
        final, classification = process_image(temp, img.copy(), 'red')
        if classification == 'stop_sign' or classification == 'red_circle':
            img_obj.add_classification(classification)
            if inp == 2:
                cv.imwrite(os.path.join(results_path, classification +
                                        's', img_obj.filename), final)
            sign_detected = True

    if sign_detected == False:
        cnt_un2 += 1
        img_obj.add_classification(classification)
        if inp == 2:
            cv.imwrite(os.path.join(
                results_path, 'unknowns', img_obj.filename), final)


#### No smoothing

In [46]:
for i in range(0, img_list.len()):
    # Get img object from img_list
    img_obj = img_list.get_image(i)

    # Get path of that image -> {type(qualquer um serve)}/{filename}
    img_path = img_obj.types[0] + 's' + os.sep + img_obj.filename

    # Image in the dataset path
    img_path_in_dataset = dataset_path + os.sep + img_path

    # Read img
    img = cv.imread(img_path_in_dataset)

    # Keep an original copy
    original = img.copy()

    # Segment it by filtering the blue and red color for blue and red signs
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')

    # Variable to check if any sign is already detected for it not to be classified as unrecognised in that case
    sign_detected = False

    # Process blue segmented img
    final, classification = process_image(img_blue.copy(), img.copy(), 'blue')

    # Process first iteration result
    if classification == 'blue_square':
        img_obj.add_classification(classification)
        if inp == 3:
            cv.imwrite(os.path.join(results_path, classification +
                                    's', img_obj.filename), final)
        sign_detected = True
    elif classification == 'unrecognised':  # if no sign detected do more smoothing
        temp = smooth(smooth(smooth(img_blue)))
        final, classification = process_image(temp, img.copy(), 'blue')
        if classification == 'blue_square':
            img_obj.add_classification(classification)
            if inp == 3:
                cv.imwrite(os.path.join(results_path, classification +
                                        's', img_obj.filename), final)
            sign_detected = True

    # Process red segmented img
    final, classification = process_image(img_red.copy(), img.copy(), 'red')

    # Process first iteration result
    if classification == 'stop_sign' or classification == 'red_circle':
        img_obj.add_classification(classification)
        if inp == 3:
            cv.imwrite(os.path.join(
                results_path, classification + 's', img_obj.filename), final)
        sign_detected = True
    elif classification == 'unrecognised':  # if no sign detected do more smoothing
        temp = smooth(img_red)
        final, classification = process_image(temp, img.copy(), 'red')
        if classification == 'stop_sign' or classification == 'red_circle':
            img_obj.add_classification(classification)
            if inp == 3:
                cv.imwrite(os.path.join(results_path, classification +
                                        's', img_obj.filename), final)
            sign_detected = True

    if sign_detected == False:
        cnt_un3 += 1
        img_obj.add_classification(classification)
        if inp == 3:
            cv.imwrite(os.path.join(
                results_path, 'unknowns', img_obj.filename), final)

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/hough.cpp:2260: error: (-211:One of the arguments' values is out of range) dp, min_dist and canny_threshold must be all positive numbers in function 'HoughCircles'


Amount of unknowns (no traffic signs detected)

In [47]:
print('Normal amount of smoothing: ' + str(cnt_un1))
print('Higher amount of smoothing: ' + str(cnt_un2))
print('No smoothing: ' + str(cnt_un3))

Normal amount of smoothing: 80
Higher amount of smoothing: 58
No smoothing: 10


### Finding a perfect circle

By using the HoughCircles function we are able to detect perfect circles in an image.

There are some cases where it fits very well but if the sign is slanted no decent circle can be found

In [ ]:
circles =  cv.HoughCircles(gray_blurred, cv.HOUGH_GRADIENT, 1.5, 100)
output = img.copy()
# ensure at least some circles were found
print(circles)
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv.circle(output, (x, y), r, (0, 255, 0), 4)
        cv.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1) #center of circle
        # show the output image
    # cv.imshow("output", np.hstack([img, output]))

# cv.waitKey(0)
# cv.destroyAllWindows()

NameError: name 'gray_blurred' is not defined

### Commented out cell

To start we need to collect all of the xml files and the corresponding png files so we can parse them.
Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

In [ ]:
# xml_files = [name for name in os.listdir('./Dataset/annotations')]
# validation_dictionary = {}

# for name in xml_files:
#     # Parse the xml
#     mytree = ET.parse('./Dataset/annotations/' + name)
    
#     # Get number from name
#     order = ''.join(i for i in name if i.isdigit())
    
#     # Get sign type
#     validation_dictionary[order] = mytree.getroot().find('object').find('name').text

# sign_types = []
# for entry in validation_dictionary:
#     if validation_dictionary[entry] not in sign_types:
#         sign_types.append(validation_dictionary[entry])
# print(sign_types)
# print(validation_dictionary)